In [ ]:
%load_ext autoreload

%autoreload 2


The following GPU devices are available: /device:GPU:0


# Check if cv2.resize collides with other stuff running on the GPU

In [24]:
from skimage.transform import resize
from sample import *

s = FeatureizedSample()
s.load("../PortionOfDataForHacking/balancedCleandDataSetBAK/positiveSamples/1.pickle")
img = s.getData(num_steps=1, imageSize=(100, 100))
print(img.shape)

_resized = resize(img, (100, 100), anti_aliasing=True)
print(_resized.shape)

(100, 100, 3)
(100, 100, 3)


In [1]:
'''
Trains a simple convnet on the MNIST dataset using a keras.utils.Sequence
to stream data, and simulate latency using `time.sleep`.
'''

from __future__ import print_function
import time

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import Sequence
import numpy as np



Using TensorFlow backend.


In [15]:
workers = 2
batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28




In [3]:
class SeqGen(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        #print("Fetching batch {}".format(idx))
        time.sleep(0.5)

        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_x, batch_y



In [4]:

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(keras.backend.image_data_format())

if keras.backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adadelta(),
    metrics=['accuracy'])

channels_last


W0707 13:30:10.431991 140408356701952 deprecation_wrapper.py:119] From /home/alubitz/anaconda3/envs/MA/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 13:30:10.455136 140408356701952 deprecation_wrapper.py:119] From /home/alubitz/anaconda3/envs/MA/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 13:30:10.459250 140408356701952 deprecation_wrapper.py:119] From /home/alubitz/anaconda3/envs/MA/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 13:30:10.522006 140408356701952 deprecation_wrapper.py:119] From /home/alubitz/anaconda3/envs/MA/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecate

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [16]:


print("Fitting using {} workers".format(workers))

model.fit_generator(
    SeqGen(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    verbose=1,
    use_multiprocessing=True,
    validation_data=(x_test, y_test),
    callbacks=[],
    workers=workers,
)


Fitting using 2 workers
Epoch 1/1
469/469 [==============================] - 120s 255ms/step - loss: 0.0543 - acc: 0.9834 - val_loss: 0.0455 - val_acc: 0.9844


In [ ]:

print("Evaluating")

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])